In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "1b583470-a522-45f4-b113-c7b764ff0d5d", #Client ID
"fs.azure.account.oauth2.client.secret": 'secretkey', #Value of secret key
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/b5a2d267-4ba4-4f75-a2c3-19ef56c332b5/oauth2/token"} #Tenant ID

dbutils.fs.mount(
source = "abfss://results@incubytestorage.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/results",
extra_configs = configs)

True

In [0]:
%fs
ls "/mnt/results"

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create Spark session
spark = SparkSession.builder.appName("PartitionData").getOrCreate()

In [0]:
# Define the connection parameters
jdbcHostname = "incubyte-pgsql.postgres.database.azure.com"
jdbcPort = 5432
jdbcDatabase = "incubyte"
jdbcUsername = "postgres"
jdbcPassword = "pwd"

# Create the JDBC URL
jdbcUrl = f"jdbc:postgresql://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"

# Set the connection properties
connectionProperties = {
    "user": jdbcUsername,
    "password": jdbcPassword,
    "driver": "org.postgresql.Driver"
}

# Read data from PostgreSQL
patients_df = spark.read.jdbc(url=jdbcUrl, table="public.patients", properties=connectionProperties)




In [0]:


# Import Dat
patients_df = patients_df.drop('H')

    # Converting Strings to Dates for better Processing
patients_df = patients_df.withColumn("customer_open_date", to_date(col("customer_open_date"),"yyyyMMdd")) \
            .withColumn("last_consulted_date", to_date(col("last_consulted_date"),"yyyyMMdd")) \
            .withColumn("dob", to_date(col("dob"),"MMddyyyy"))

    # Adding Additional Derivative Columns
    # Add Age
patients_df = patients_df.withColumn("Age",(datediff(current_date(), col("dob")) / 365.25).cast("int"))

    # days since last consulted >30 
patients_df = patients_df.withColumn("Days_Since_Last_Consulted_>_30", when((datediff(current_date(),"last_consulted_date").cast("int")) >= 30, True).otherwise(False))


In [0]:
patients_df.toPandas().head()

,headerrow,customer_name,customer_id,customer_open_date,last_consulted_date,vaccination_type,doctor_consulted,state,country,dob,active_customer,Age,Days_Since_Last_Consulted_>_30
0,D,John,953405,2010-03-12,2015-04-27,MMR,Smith ...,WAS,IND,1977-03-10,A,47,True
1,D,Emily,449792,2019-01-24,2015-01-31,FLU,Smith ...,WAS,CAN,1987-04-08,A,37,True
2,D,Matt,763362,2020-01-20,2017-08-01,MMR,John ...,TN,IND,1972-09-29,I,52,True
3,D,Emily,645853,2010-06-19,2017-12-07,HPT,Smith ...,TN,NYC,1961-01-07,I,63,True
4,D,John,754585,2019-05-24,2016-03-26,MVD,John ...,VIC,AU,1964-04-19,I,60,True


In [0]:
patients_df.write.format("csv").mode("overwrite").partitionBy("country").save("/mnt/results/partitionedfiles")

In [0]:
%fs
ls "/mnt/results/partitionedfiles"

path,name,size,modificationTime
dbfs:/mnt/results/partitionedfiles/_SUCCESS,_SUCCESS,0,1728885654000
dbfs:/mnt/results/partitionedfiles/country=AU /,country=AU /,0,1728885651000
dbfs:/mnt/results/partitionedfiles/country=CAN /,country=CAN /,0,1728885652000
dbfs:/mnt/results/partitionedfiles/country=FRA /,country=FRA /,0,1728885652000
dbfs:/mnt/results/partitionedfiles/country=GER /,country=GER /,0,1728885652000
dbfs:/mnt/results/partitionedfiles/country=IND /,country=IND /,0,1728885652000
dbfs:/mnt/results/partitionedfiles/country=JPN /,country=JPN /,0,1728885653000
dbfs:/mnt/results/partitionedfiles/country=NYC /,country=NYC /,0,1728885653000
dbfs:/mnt/results/partitionedfiles/country=PHIL /,country=PHIL /,0,1728885653000
dbfs:/mnt/results/partitionedfiles/country=UK /,country=UK /,0,1728885653000
